In [2]:
import pandas as pd
import numpy as np
#from spellchecker import SpellChecker
import re
import string
from nltk.corpus import stopwords
from tqdm import tqdm

In [3]:
def contraction_count(sent):
    count = 0
    count += re.subn(r"won\'t", "", sent)[1]
    count += re.subn(r"can\'t", "", sent)[1]
    count += re.subn(r"\'re", "", sent)[1]
    count += re.subn(r"\'s", "", sent)[1]
    count += re.subn(r"\'d", "", sent)[1]
    count += re.subn(r"\'ll", "", sent)[1]
    count += re.subn(r"\'t", "", sent)[1]
    count += re.subn(r"\'ve", "", sent)[1]
    count += re.subn(r"\'m", "", sent)[1]
    return count

def text_features(df, col):
    df[f"{col}_num_words"] = df[col].apply(
        lambda x: len(str(x).split())
    )  # num_words count

    df[f"{col}_num_unique_words"] = df[col].apply(
        lambda x: len(set(str(x).split()))
    )  # num_unique_words count

    df[f"{col}_num_chars"] = df[col].apply(lambda x: len(str(x)))  # num_chars count

    df[f"{col}_num_stopwords"] = df[col].apply(
        lambda x: len(
            [w for w in str(x).lower().split() if w in stopwords.words("english")]
        )
    )  # stopword count 冠詞

    df[f"{col}_num_punctuations"] = df[col].apply(
        lambda x: len([c for c in str(x) if c in list(string.punctuation)])
    )  # num_punctuations count 句読点

    df[f"{col}_num_words_upper"] = df[col].apply(
        lambda x: len([w for w in str(x) if w.isupper()])
    )  # num_words_upper count 大文字

    df[f"{col}_mean_word_len"] = df[col].apply(
        lambda x: np.mean([len(w) for w in x.split()])
    )  # mean_word_len

    df[f"{col}_num_paragraphs"] = df[col].apply(
        lambda x: len(list(filter(None, re.split("\n\n|\r\n|\r|\n", x))))
    )  # num_paragraphs count

    df[f"{col}_num_contractions"] = df[col].apply(
        contraction_count
    )  # num_contractions count

    return df


def check_spell(df, col):
    spell = SpellChecker()
    df["misspelled"] = 0
    for idx, text in enumerate(tqdm(df[col])):
        text = re.split(
            '[\n\n|\r\n|\r|\n| |!|#|$|%|&|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|\|^|_|`|{|}|~|"|\||\[|\]]',
            text,
        )
        misspelled = spell.unknown(text)
        if "" in misspelled:
            misspelled.remove("")
        df.loc[idx, "misspelled"] = len(misspelled)

    return df

In [4]:
# read csv
train = pd.read_csv("../../data/input/train.csv")

In [ ]:
train = text_features(train, "full_text")
train = check_spell(train, "full_text")

In [11]:
train_X.to_csv(f"../../data/input/train_with_meta.csv", index=False)